In [1]:
from sentinel_tiles import sentinel_tiles
import global_forecasting_system
from global_forecasting_system import forecast_Ta_C

In [2]:
time_UTC = "2025-09-30 21:11:22.358885"

In [3]:
geometry = sentinel_tiles.grid("11SPS")
geometry

{
  "dimensions": {
    "rows": 3660,
    "cols": 3660
  },
  "bbox": {
    "xmin": 600000.0,
    "ymin": 3590220.0,
    "xmax": 709800.0,
    "ymax": 3700020.0
  },
  "crs": "EPSG:32611",
  "resolution": {
    "cell_width": 30.0,
    "cell_height": -30.0
  }
}

In [4]:
from global_forecasting_system import earliest_time_UTC

earliest = earliest_time_UTC()
earliest

ValueError: zero-length GFS listing

In [ ]:
earliest.to_pydatetime().date()

In [ ]:
type(earliest)

In [ ]:
Ta_C = forecast_Ta_C(
    time_UTC=time_UTC,
    geometry=geometry,
)

Ta_C

In [ ]:
listing = global_forecasting_system.get_GFS_listing()
listing

In [ ]:
listing[listing.forecast_time_UTC.apply(lambda forecast_time_UTC: str(forecast_time_UTC) <= str(time_UTC))]

In [ ]:
before = listing[listing.forecast_time_UTC.apply(lambda forecast_time_UTC: str(forecast_time_UTC) <= str(time_UTC))].iloc[[-1]]

In [ ]:
# Let's debug the issue by examining the data
from global_forecasting_system import GFS_file_addresses
import pandas as pd
import posixpath
from datetime import datetime
from os.path import splitext

# Get some sample data to see the issue
try:
    addresses = GFS_file_addresses()[:10]  # Just get first 10 for debugging
    address_df = pd.DataFrame({"address": addresses})
    address_df["basename"] = address_df.address.apply(lambda address: posixpath.basename(address))
    print("Sample basenames:")
    print(address_df["basename"].head())
    
    print("\nTrying to extract source_hour:")
    address_df["source_hour"] = address_df.basename.apply(lambda basename: int(basename.split("_")[3][:2]))
    print("Source hours:")
    print(address_df["source_hour"])
    print("Source hour dtypes:")
    print(address_df["source_hour"].dtype)
    
except Exception as e:
    print(f"Error during debugging: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Let's test the GFS most recent date address function
from global_forecasting_system import GFS_most_recent_date_address

try:
    date_url = GFS_most_recent_date_address()
    print(f"Date URL: {date_url}")
    
    # Test if we can fetch the page
    import requests
    response = requests.get(date_url)
    print(f"Response status: {response.status_code}")
    print(f"Response length: {len(response.text)}")
    
    if response.status_code == 200:
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")
        links = [link.get("href") for link in soup.find_all("a")]
        print(f"Total links found: {len(links)}")
        
        # Look for .grb2 files
        from fnmatch import fnmatch
        grb2_files = [link for link in links if fnmatch(link, "*.grb2")]
        print(f"GRB2 files found: {len(grb2_files)}")
        if len(grb2_files) > 0:
            print("Sample GRB2 files:")
            for i, file in enumerate(grb2_files[:5]):
                print(f"  {i+1}: {file}")
        
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Reload the module to get the latest changes
import importlib
import global_forecasting_system
importlib.reload(global_forecasting_system)

# Test the updated function
from global_forecasting_system import earliest_time_UTC

try:
    earliest = earliest_time_UTC()
    print(f"Success: {earliest}")
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Test again with fresh import
from global_forecasting_system import get_GFS_listing

try:
    listing = get_GFS_listing()
    print(f"GFS listing length: {len(listing)}")
    print(f"Columns: {listing.columns.tolist()}")
    print(f"Dtypes: {listing.dtypes}")
    
    if len(listing) == 0:
        print("Empty listing returned successfully!")
    else:
        print("Non-empty listing:")
        print(listing.head())
        
except Exception as e:
    print(f"Error in get_GFS_listing: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Test the earliest_time_UTC function
from global_forecasting_system import earliest_time_UTC

try:
    earliest = earliest_time_UTC()
    print(f"Success: {earliest}")
except ValueError as e:
    print(f"Expected ValueError: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")
    import traceback
    traceback.print_exc()